In [ ]:
!pip install ultralytics
!pip install pycuda

import pycuda.driver as cuda
import os
from ultralytics import YOLO
from IPython.display import display, Image
from IPython.display import Image as IPImage
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

In [ ]:
DATASET_PATH = '/kaggle/input/facial-feature-extraction-dataset'
DATA_YAML = os.path.join(DATASET_PATH, 'Facial Feature Extraction Dataset/data.yaml')

In [ ]:
!nvidia-smi

In [ ]:
# !pip show accelerate
# !pip install git+https://github.com/huggingface/accelerate

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

!source ~/.bashrc

In [ ]:
class FeatureExtractionModel:
    def __init__(self, yolo_model_name):
        self.model = YOLO(yolo_model_name)
        
    def train_model(self):
        """
        Initialize and train YOLOv8 model with specified parameters
        Returns:
            training results
        """
        cuda.init()
        device_count = cuda.Device.count()
        device = ','.join(str(i) for i in range(device_count)) if device_count > 0 else 'cpu'

        # Initialize the wandb logging before training
        # wandb.init(project="yolo_training", config={"epochs": 20, "batch_size": 16})
        results = self.model.train(
            data=DATA_YAML,       # data.yaml file from Roboflow
            epochs=20,            # number of epochs
            imgsz=640,            # image size
            batch=32,             # batch size
            name='yolov8_custom', # folder name for training results
            device=device,        # '0' for GPU, 'cpu' for CPU
            patience=50,          # early stopping patience
            save=True,            # save best model
            pretrained=True,      # use pretrained weights
            plots=True,           # save training plots
            cache=True,           # enable caching
            verbose=True,         # Enable verbose logging
            workers=32
        )
        
        return results
    
    def evaluate_model(self, model_path):
        """
        Evaluate trained model using validation dataset
        Args:
            model_path: path to the trained model weights
        Returns:
            evaluation metrics
        """
        model = YOLO(model_path)
        metrics = model.val()
        return metrics
    
    def visualize_metrics(self, metrics):
        """
        Create visualizations for model performance metrics
        Args:
            metrics: model evaluation metrics
        """
        # Create a DataFrame for metrics
        metrics_dict = {
            'Metric': ['Precision', 'Recall', 'mAP50', 'mAP50-95'],
            'Value': [
                metrics.results_dict['metrics/precision(B)'],
                metrics.results_dict['metrics/recall(B)'],
                metrics.results_dict['metrics/mAP50(B)'],
                metrics.results_dict['metrics/mAP50-95(B)']
            ]
        }
        df_metrics = pd.DataFrame(metrics_dict)
        
        # Create bar plot
        plt.figure(figsize=(10, 6))
        sns.barplot(data=df_metrics, x='Metric', y='Value')
        plt.title('Model Performance Metrics')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Display class-wise mAP
        print("\nClass-wise mAP Scores:")
        for class_name, map_score in zip(metrics.names.values(), metrics.maps):
            print(f"{class_name}: {map_score:.3f}")
            

In [ ]:
def main():
    # Verify data path
    if not os.path.exists(DATA_YAML):
        raise FileNotFoundError(f"data.yaml file not found: {DATA_YAML}")
    
    print("🚀 Training started...")
    print(f"📁 Dataset path: {DATASET_PATH}")
    
    # Train model
    model = FeatureExtractionModel('yolov8m.pt')
    model.train_model()
    
    # Evaluate best model
    best_model_path = f'runs/detect/yolov8_custom/weights/best.pt'
    metrics = model.evaluate_model(best_model_path)
    
    print("\n✨ Training completed!")
    print(f"💾 Best model saved at: {best_model_path}")
    
    print("\n📊 Model Performance Summary:")
    print("=" * 50)
    print(f"Precision: {metrics.results_dict['metrics/precision(B)']:.3f}")
    print(f"Recall: {metrics.results_dict['metrics/recall(B)']:.3f}")
    print(f"mAP50: {metrics.results_dict['metrics/mAP50(B)']:.3f}")
    print(f"mAP50-95: {metrics.results_dict['metrics/mAP50-95(B)']:.3f}")
    print("=" * 50)
    

In [ ]:
if __name__ == "__main__":
    main()
    # log missing due to multi GPU training